<a href="https://colab.research.google.com/github/chewinggumtnp-cmd/TapMelon-to-Ripe/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os # Import os module for directory operations
import numpy as np
import librosa

# --- 1. กำหนดค่าพื้นฐาน ---
IMG_HEIGHT = 224  # ย่อรูปจาก 1892 เหลือ 224
IMG_WIDTH = 224  # ย่อรูปจาก 872 เหลือ 224
BATCH_SIZE = 8  # ข้อมูลน้อย ใช้ Batch เล็กๆ เพื่อให้ AI ค่อยๆ เรียนรู้
DATA_DIR = 'watermelon_dataset'  # ชื่อโฟลเดอร์ที่เก็บรูป (ต้องมีโฟลเดอร์ย่อย: ripe, mid, unripe)

# --- มีข้อมูลแล้ว

# สร้างไฟล์ภาพจำลองในแต่ละโฟลเดอร์
# ต้องสร้างไฟล์ที่ Keras สามารถอ่านได้ เช่น .jpg หรือ .png
# ในที่นี้จะสร้างไฟล์เปล่าๆ แต่ Keras image_dataset_from_directory อาจต้องการไฟล์ที่มีข้อมูลภาพจริง
# สำหรับการทดสอบนี้ อาจต้องใช้ Pillow หรือไลบรารีอื่นช่วยสร้างภาพจริง
# แต่เพื่อไม่ให้ซับซ้อนเกินไป จะสร้างไฟล์ dummy .txt แทน (ซึ่งจะทำให้เกิด error อีกหาก Keras พยายามอ่าน)
# ทางที่ดีที่สุดคือให้ผู้ใช้อัปโหลดข้อมูลภาพจริง

# Ensure the base data directory exists
os.makedirs(DATA_DIR, exist_ok=True)

# Ensure subdirectories for each class exist before attempting to save files
for label_folder in ['ripe', 'mid', 'unripe']:
    os.makedirs(os.path.join(DATA_DIR, label_folder), exist_ok=True)

# เพื่อให้โค้ดทำงานได้จริงโดยไม่เกิด NotFoundError แต่ยังอาจเกิด InvalidArgumentError จากภาพเปล่า
# ผมจะสร้างไฟล์ภาพจริงขนาดเล็กมากๆ ด้วย Pillow หาก Pillow มีในสภาพแวดล้อม
try:
    from PIL import Image
    dummy_image = Image.new('RGB', (IMG_WIDTH, IMG_HEIGHT), color = 'red')
    for i in range(50): # สร้าง 50 รูปต่อคลาส สำหรับการทดสอบ
        dummy_image.save(os.path.join(DATA_DIR, 'ripe', f'dummy_ripe_{i}.jpg'))
        dummy_image.save(os.path.join(DATA_DIR, 'mid', f'dummy_mid_{i}.jpg'))
        dummy_image.save(os.path.join(DATA_DIR, 'unripe', f'dummy_unripe_{i}.jpg'))
    print(f"สร้างไฟล์ภาพจำลองใน '{DATA_DIR}' เรียบร้อยแล้ว (โดยใช้ Pillow).")
except ImportError:
    print("Pillow ไม่ได้ติดตั้ง. ไม่สามารถสร้างไฟล์ภาพจำลองได้. โปรดอัปโหลดไฟล์ภาพจริง.")
    # Fallback to create dummy .txt files. Directories are already created above.
    open(os.path.join(DATA_DIR, 'ripe', 'dummy.txt'), 'a').close()
    open(os.path.join(DATA_DIR, 'mid', 'dummy.txt'), 'a').close()
    open(os.path.join(DATA_DIR, 'unripe', 'dummy.txt'), 'a').close()
    print(f"สร้างไฟล์ .txt จำลองใน '{DATA_DIR}' เพื่อหลีกเลี่ยง NotFoundError (แต่จะเกิด error อื่นแทน).")

import os

# เช็คว่าเจอไฟล์กี่ไฟล์ในแต่ละคลาส
for label in ['ripe', 'mid', 'unripe']:
    path = os.path.join(DATA_DIR, label)
    if os.path.exists(path):
        count = len(os.listdir(path))
        print(f"คลาส {label}: พบรูปภาพ {count} รูป")
    else:
        print(f"หาโฟลเดอร์ {label} ไม่เจอ! ตรวจสอบ Path ด้วยครับ")

# --- 2. โหลดข้อมูลและเตรียมรูปภาพ ---
# บรรทัดนี้จะอ่านไฟล์ ย่อขนาด และแบ่งข้อมูลเรียน/สอบ ให้ทันที
try:
    train_ds = tf.keras.utils.image_dataset_from_directory(
        DATA_DIR,
        validation_split=0.2,  # แบ่ง 20% ไว้ทดสอบ (ประมาณ 72 รูป)
        subset="training",
        seed=123,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE
    )

    val_ds = tf.keras.utils.image_dataset_from_directory(
        DATA_DIR,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE
    )
except Exception as e:
    print(f"เกิดข้อผิดพลาดในการโหลดข้อมูล: {e}")
    print("โปรดตรวจสอบว่าโฟลเดอร์ 'watermelon_dataset' มีโครงสร้างที่ถูกต้อง (มีโฟลเดอร์ย่อย ripe, mid, unripe) และมีไฟล์ภาพอยู่ในนั้นจริง")
    # หยุดการทำงานของโค้ดส่วนที่เหลือหากโหลดข้อมูลไม่ได้
    raise

#ข้ามย้าย 3.เพิ่มจำนวนข้อมูลแทรกใน 4

# --- 4. สร้างโมเดล CNN (แทรก Data Augmentation ไว้ที่นี่) ---

model = tf.keras.Sequential([
    # --- ใส่ Data Augmentation เป็นส่วนแรกของโมเดล ---
    layers.RandomFlip("horizontal", input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),

    # --- จากนั้นตามด้วย Layer ปกติ ---
    layers.Rescaling(1./255), # ปรับค่าสี

    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),

    # บรรทัดสุดท้ายต้องเป็น 3 ตามจำนวนคลาส (ripe, mid, unripe)
    layers.Dense(3)
])

# --- 5. คอมไพล์โมเดล ---
# กำหนด Optimizer, Loss function และ Metrics สำหรับการเทรน
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# --- 6. เริ่มการเทรน (Training) ---
epochs = 30  # ลองเทรน 30 รอบก่อน
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

# แบบเดิม (ที่ขึ้น Warning)
# model.save('watermelon_model.h5')

# แบบใหม่ที่แนะนำ (Warning จะหายไป)
model.save('watermelon_model.keras')
print("โมเดลถูกบันทึกเรียบร้อยแล้ว!")

from tensorflow.keras.models import load_model

# โหลดไฟล์ .keras ที่บันทึกไว้
model = load_model('watermelon_model.keras')

# โหลดโมเดลที่บันทึกไว้
# prepare_audio function needs to be defined BEFORE it is used.
#เตรียมข้อมูลใหม่ (Preprocessing Pipeline)
def prepare_audio(file_path):
    # 1. โหลดเสียง
    y, sr = librosa.load(file_path, duration=1.0) # เอาแค่ 1 วินาทีเท่าตอนเทรน
    # 2. แปลงเป็น Mel Spectrogram
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    S_dB = librosa.power_to_db(S, ref=np.max)
    # 3. ปรับขนาดภาพ (Resize) และ Normalization
    # ปรับขนาดภาพให้เป็นขนาดเดียวกับที่โมเดลคาดหวัง (IMG_HEIGHT, IMG_WIDTH)
    img = tf.image.resize(S_dB[..., np.newaxis], [IMG_HEIGHT, IMG_WIDTH])
    # ทำซ้ำแชนเนลเดียวเป็น 3 แชนเนล เพื่อให้ตรงกับอินพุตของโมเดลที่ฝึกกับรูปภาพ RGB
    img = tf.concat([img, img, img], axis=-1)
    img = img / 255.0  # ทำเหมือนตอนเทรน
    # 4. เพิ่มมิติ Batch (เป็น [1, IMG_HEIGHT, IMG_WIDTH, 3])
    return np.expand_dims(img, axis=0)

#สั่งทำนายผล (Prediction)
# ใส่ที่อยู่ไฟล์เสียงที่ต้องการทดสอบ
new_audio = prepare_audio('watermelon_unripe_3_1.wav')

# 1. ทำนายผล
predictions = model.predict(new_audio)

# 2. ใช้ Softmax เพื่อแปลงค่าให้เป็นความน่าจะเป็น (Probability) 0-100%
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# แปลงค่า predictions ของใบแรก [0]
prob = softmax(predictions[0])

classes = ['Mid', 'Ripe', 'Unripe']

# 3. ดึง Index ที่ค่าสูงที่สุด
result_index = np.argmax(prob)

# 4. แสดงผล
print(f"ผลทำนาย: {classes[result_index]}")
print(f"ความมั่นใจ: {prob[result_index]*100:.2f}%")
print(f"ค่า Probability ทั้งหมด: {prob}") # จะเห็นว่ารวมกันได้ 1.0 พอดี


# โหลดโมเดล (ระบุชื่อไฟล์ที่คุณบันทึกไว้)
model = tf.keras.models.load_model('watermelon_model.keras')


#สั่งทำนายผล (Prediction)
# ใส่ที่อยู่ไฟล์เสียงที่ต้องการทดสอบ
new_audio = prepare_audio('watermelon_unripe_3_1.wav')

# ทำนาย
predictions = model.predict(new_audio)
classes = ['Mid', 'Ripe', 'Unripe']

# ดึงค่าที่มั่นใจที่สุดออกมา
result_index = np.argmax(predictions)
print(f"ผลทำนาย: {classes[result_index]} (ความมั่นใจ: {predictions[0][result_index]*100:.2f}%) errors: {predictions[0]}")

import streamlit as st
from st_audiorec import st_audiorec
import librosa
import numpy as np
import tensorflow as tf # นำเข้า TensorFlow

# 1. โหลดโมเดล Keras ที่อัปโหลดไว้บน GitHub
# ตรวจสอบให้แน่ใจว่าชื่อไฟล์ตรงกัน (เช่น 'watermelon_model.keras')
try:
    model = tf.keras.models.load_model('watermelon_model.keras')
    st.success("โหลดโมเดลสำเร็จแล้ว!")
except Exception as e:
    st.error(f"ไม่สามารถโหลดโมเดลได้: {e}")
    st.stop() # หยุดการทำงานหากโหลดโมเดลไม่ได้

st.title("TapMelon-to-Ripe")
st.write("กดปุ่มด้านล่างเพื่ออัดเสียงเคาะแตงโม แล้วระบบจะทำนายความสุก")

wav_audio_data = st_audiorec()

if wav_audio_data is not None:
    st.audio(wav_audio_data, format='audio/wav')
    st.info("กำลังวิเคราะห์เสียง...")

    # 2. เตรียมข้อมูลเสียงให้อยู่ในรูปแบบที่โมเดลต้องการ
    # บันทึกข้อมูลเสียงชั่วคราวเป็นไฟล์ .wav เพื่อให้ librosa อ่านได้
    with open("temp_audio.wav", "wb") as f:
        f.write(wav_audio_data)

    # โหลดเสียงด้วย librosa
    y, sr = librosa.load("temp_audio.wav", sr=22050) # ปรับ sr ให้ตรงกับที่ฝึกโมเดลมา

    # สกัดคุณสมบัติ (ปรับแต่งตามที่คุณใช้ฝึกโมเดลมา)
    # ตัวอย่าง: สกัด MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    features = np.mean(mfccs.T, axis=0)

    # Keras ต้องการ input เป็น batch (หลายตัวอย่าง) เลยต้องเพิ่มมิติ
    input_features = np.expand_dims(features, axis=0)

    # 3. ทำนายผล
    prediction = model.predict(input_features)

    # 4. แสดงผลลัพธ์
    # ปรับ logic นี้ตาม output จริงของโมเดลคุณ
    if prediction[0][0] > 0.5:
        result = "สุกแล้ว! 🎉"
    else:
        result = "ยังไม่สุก  unripe 😕"

    st.metric("ผลการทำนายระดับความสุก", result)



สร้างไฟล์ภาพจำลองใน 'watermelon_dataset' เรียบร้อยแล้ว (โดยใช้ Pillow).
คลาส ripe: พบรูปภาพ 50 รูป
คลาส mid: พบรูปภาพ 50 รูป
คลาส unripe: พบรูปภาพ 50 รูป
Found 150 files belonging to 3 classes.
Using 120 files for training.
Found 150 files belonging to 3 classes.
Using 30 files for validation.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.2835 - loss: 9.0427 - val_accuracy: 0.3667 - val_loss: 1.1394
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.2401 - loss: 1.1460 - val_accuracy: 0.2000 - val_loss: 1.1028
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.3760 - loss: 1.0975 - val_accuracy: 0.2000 - val_loss: 1.1083
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.3467 - loss: 1.0975 - val_accuracy: 0.2000 - val_loss: 1.1088
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.3800 - loss: 1.0964 - val_accuracy: 0.2000 - val_loss: 1.1124
Epoch 6/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.3396 - loss: 1.1003 - val_accuracy: 0.2000 - val_loss: 1.1134
Epoch 7/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.4168 - loss: 1.0915 - val_accuracy: 0.2000 - val_loss: 1.1198
Epoch 8/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.3485 - loss: 1.0991 - val_accuracy: 0.2000 - val_loss: